In [1]:
#setting up
import boto3, re
from sagemaker import get_execution_role

role = get_execution_role()

In [2]:
import keras
from keras.models import model_from_json
import tensorflow as tf

tf.compat.v1.disable_eager_execution()

Using TensorFlow backend.


In [3]:
!mkdir keras_model

mkdir: cannot create directory ‘keras_model’: File exists


In [4]:
#loading the model
json_file = open('keras_model/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json,custom_objects={"GlorotUniform": tf.keras.initializers.glorot_uniform})




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:
#loading the model weights
loaded_model.load_weights('keras_model/model_weights.h5')
print("Loaded model from disk")







Loaded model from disk


In [7]:
from tensorflow.python.saved_model import builder
from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
from tensorflow.python.saved_model import tag_constants

# Note: This directory structure will need to be followed - see notes for the next section

model_version = '1'
export_dir = 'export/Servo/' + model_version
export_dir

'export/Servo/1'

In [8]:
import shutil
shutil.rmtree(export_dir)

In [9]:
# Build the Protocol Buffer SavedModel at 'export_dir'
build = builder.SavedModelBuilder(export_dir)

In [10]:
# Create prediction signature to be used by TensorFlow Serving Predict API
signature = predict_signature_def(
    inputs={"inputs": loaded_model.input}, outputs={"score": loaded_model.output})

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


In [11]:
from keras import backend as K

with K.get_session() as sess:
    # Save the meta graph and variables
    build.add_meta_graph_and_variables(
        sess=sess, tags=[tag_constants.SERVING], signature_def_map={"serving_default": signature})
    build.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: export/Servo/1/saved_model.pb


In [12]:
!ls export/Servo/variables

ls: cannot access export/Servo/variables: No such file or directory


In [13]:
import tarfile
with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add('export', recursive=True)

In [14]:
import sagemaker

sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='model')

In [15]:
!touch train.py

In [17]:
from sagemaker.tensorflow.model import TensorFlowModel
sagemaker_model = TensorFlowModel(model_data = 's3://' + sagemaker_session.default_bucket() + '/model/model.tar.gz',
                                  role = role,
                                  framework_version = '1.12',
                                  entry_point = 'train.py')

In [20]:
%%time
predictor = sagemaker_model.deploy(initial_instance_count=1,
                                   instance_type='ml.t2.medium')

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


-------------!CPU times: user 1.16 s, sys: 122 ms, total: 1.29 s
Wall time: 6min 33s


In [21]:
predictor.endpoint

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


'sagemaker-tensorflow-serving-2021-10-15-00-27-55-675'

In [ ]:
endpoint_name = 'sagemaker-tensorflow-serving-2021-10-15-00-27-55-675'